In [24]:
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'TRUE'
import torch
from torch.utils import data
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d
import random
import numpy as np
%matplotlib inline

In [25]:
# 构建人造数据集
def synthetic_data(w, b, num_examples): 
    # 生成 y = Xw + b + 噪声
    X = torch.normal(0, 1, (num_examples, len(w))) # normal(mean, std, generator) 生成正态分布随机数，mean表示正态分布的均值，std表示正态分布的标准差，generator是随机种子
    y = torch.matmul(X, w) + b # matmul适用于高维张量
    y += torch.normal(0, 0.01, y.shape)
    return X, y.reshape((-1, 1)) #reshape确保y是列向量,-1表示自动分配行数

true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = synthetic_data(true_w, true_b, 1000)

print(features)
print(labels)

tensor([[ 1.6302,  0.6250],
        [-1.2596, -1.7367],
        [ 1.3685,  2.0619],
        ...,
        [ 1.0050,  0.4381],
        [-0.2036,  0.2038],
        [ 1.5125, -1.7046]])
tensor([[ 5.3564e+00],
        [ 7.5919e+00],
        [-8.3524e-02],
        [ 1.1440e+01],
        [ 7.6274e+00],
        [ 7.5191e+00],
        [-1.3673e+00],
        [ 3.8555e+00],
        [ 7.2210e+00],
        [-4.2948e+00],
        [-4.8152e+00],
        [ 2.6468e+00],
        [ 7.1373e+00],
        [ 5.9552e+00],
        [ 5.1717e-01],
        [ 2.4917e+00],
        [ 3.9900e+00],
        [ 3.6351e+00],
        [ 1.0241e+01],
        [ 5.6594e+00],
        [-3.8781e+00],
        [-1.3781e+00],
        [ 5.3151e+00],
        [-1.1359e+00],
        [ 7.0869e+00],
        [ 2.4105e+00],
        [ 6.2269e+00],
        [ 4.9424e+00],
        [ 7.0990e+00],
        [ 9.7613e+00],
        [ 3.1125e+00],
        [-1.7453e-02],
        [ 2.0340e-01],
        [-5.3783e+00],
        [ 1.8244e+00],
        [ 3.7

In [26]:
def load_array(data_array, batch_size, is_train= True):
    '''构造一个PyTorch数据迭代器'''
    dataset = data.TensorDataset(*data_array) # *代表给元组解包
    return data.DataLoader(dataset, batch_size, shuffle= is_train)

batch_size = 10
data_iter = load_array((features, labels), batch_size)

next(iter(data_iter))

[tensor([[-2.1833,  0.1592],
         [ 0.8525, -1.6109],
         [ 0.1124,  0.6568],
         [-0.1591,  2.7201],
         [ 0.7304, -1.2987],
         [-1.7798,  0.7692],
         [-1.3013,  0.2338],
         [-1.0664, -0.0517],
         [-1.1461,  1.1636],
         [-1.0482, -0.3280]]),
 tensor([[-0.6929],
         [11.3822],
         [ 2.2134],
         [-5.3783],
         [10.0937],
         [-1.9813],
         [ 0.7908],
         [ 2.2371],
         [-2.0505],
         [ 3.2153]])]

In [27]:
from torch import nn # nn = neural net

net = nn.Sequential(nn.Linear(2, 1))

In [34]:
# 初始化模型参数

net[0].weight.data.normal_(0, 0.01)
net[0].bias.data.fill_(0)

print(net[0].weight)
print(net[0].bias)

Parameter containing:
tensor([[-0.0192, -0.0013]], requires_grad=True)
Parameter containing:
tensor([0.], requires_grad=True)


In [35]:
loss = nn.MSELoss()

In [36]:
trainer = torch.optim.SGD(net.parameters(), lr= 0.03)

In [37]:
num_epochs = 3
for epoch in range(num_epochs):
    for X, y in data_iter:
        l = loss(net(X), y)
        trainer.zero_grad()
        l.backward()
        trainer.step()

    l = loss(net(features), labels)
    print(f"epoch{epoch}, loss:{l}")
    print(net[0].weight)
    print(net[0].bias)
    print("\n")

epoch0, loss:0.0003347343881614506
Parameter containing:
tensor([[ 1.9935, -3.3916]], requires_grad=True)
Parameter containing:
tensor([4.1878], requires_grad=True)


epoch1, loss:0.00010146367276320234
Parameter containing:
tensor([[ 1.9995, -3.4008]], requires_grad=True)
Parameter containing:
tensor([4.1988], requires_grad=True)


epoch2, loss:0.00010131667659152299
Parameter containing:
tensor([[ 1.9992, -3.3995]], requires_grad=True)
Parameter containing:
tensor([4.2004], requires_grad=True)


